Here, the data is loaded into a pandas dataframe from my Google drive. The data is posted with the paper by Hou et al., at https://translational-medicine.biomedcentral.com/articles/10.1186/s12967-020-02620-5#availability-of-data-and-materials

In [2]:
import pandas as pd

sepsis_df = pd.read_csv("/content/drive/MyDrive/CS 598 - Deep Learning for Healthcare/Final Project/Reproducing Septic Paper/sepsis_data.csv")

Here, we can see the columns from this dataset, which include demographic information, vital signs, and lab results.

In [3]:
for col in sepsis_df.columns:
  print(col)

icustay_id
hadm_id
intime
outtime
dbsource
suspected_infection_time_poe
suspected_infection_time_poe_days
specimen_poe
positiveculture_poe
antibiotic_time_poe
blood_culture_time
blood_culture_positive
age
gender
is_male
ethnicity
race_white
race_black
race_hispanic
race_other
metastatic_cancer
diabetes
first_service
hospital_expire_flag
thirtyday_expire_flag
icu_los
hosp_los
sepsis_angus
sepsis_martin
sepsis_explicit
septic_shock_explicit
severe_sepsis_explicit
sepsis_nqf
sepsis_cdc
sepsis_cdc_simple
elixhauser_hospital
vent
sofa
lods
sirs
qsofa
qsofa_sysbp_score
qsofa_gcs_score
qsofa_resprate_score
aniongap_min
aniongap_max
bicarbonate_min
bicarbonate_max
creatinine_min
creatinine_max
chloride_min
chloride_max
glucose_min
glucose_max
hematocrit_min
hematocrit_max
hemoglobin_min
hemoglobin_max
lactate_min
lactate_max
lactate_mean
platelet_min
platelet_max
potassium_min
potassium_max
inr_min
inr_max
sodium_min
sodium_max
bun_min
bun_max
bun_mean
wbc_min
wbc_max
wbc_mean
heartrate_min
he

We confirm that there are no missing values in the target, which is thirtyday_expire_flag, indicating whether or not the patient died within 30 days.

In [4]:
sepsis_df["thirtyday_expire_flag"].isna().sum()

0

We look at the value counts for the target, seeing that there are 3670 patients who survived, and 889 patients who died within 30 days. We see that we have a class imbalance, with 80.5% of patients surviving.

In [5]:
sepsis_df["thirtyday_expire_flag"].value_counts()

0    3670
1     889
Name: thirtyday_expire_flag, dtype: int64

Now, we select the target.

In [6]:
y = sepsis_df["thirtyday_expire_flag"]

Next, we drop columns that are not used for prediction. Note that gender and ethnicity are one-hot encoded, and these one-hot encoded variables are still included in the dataset.

In [7]:
to_drop = ["icustay_id",
           "hadm_id",
           "intime",
           "outtime",
           "dbsource",
           "suspected_infection_time_poe",
           "suspected_infection_time_poe_days",
           "specimen_poe",
           "positiveculture_poe",
           "antibiotic_time_poe",
           "blood_culture_time",
           "blood_culture_positive",
           "gender",
           "ethnicity",
           "hospital_expire_flag",
           "thirtyday_expire_flag",
           "first_service"]

X = sepsis_df.drop(columns = to_drop)

We are now left with only numerical features. We perform mean value imputation on all columns to fill in missing values.

In [8]:
X = X.apply(lambda x:x.fillna(x.mean()), axis = 0).values

Now, we ust the StandardScaler from SciKit-Learn to normalize the data so that each feature has mean 0 and standard deviation 1.

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

We split the data into a training set and a testing set, which we will use to train and evaluate models. The training set is 80% of the data, while the testing set is 20% of the data.

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, stratify = y, random_state = 0)

We will use a memory profile to check the computational requirements.

In [11]:
!pip install memory_profiler
%load_ext memory_profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31284 sha256=b2af9f2d3bec475fa4828a9fd4d5a80aa6c447fa38c9888bc18dc8958dbb6aa3
  Stored in directory: /root/.cache/pip/wheels/67/2b/fb/326e30d638c538e69a5eb0aa47f4223d979f502bbdb403950f
Successfully built memory-profiler


Next, we use a CNN. We first convert the data to tensors, and make the data loaders.

In [12]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

batch_size = 64

X_train = torch.Tensor(X_train)
y_train = torch.LongTensor(y_train.values)

X_train = torch.unsqueeze(X_train, 1)

train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size)

X_test = torch.Tensor(X_test)
y_test = torch.LongTensor(y_test.values)

X_test = torch.unsqueeze(X_test, 1)

test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size)

Here, we define the CNN. Modifications from the paper by Perng et al. are using kernels sizes of 3, and changing the number of neurons in the first fully connected layer to 1024.

In [23]:
# Define model
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 8, 3, padding = 1)   # in channels, out channels, kernel size
        self.conv2 = nn.Conv1d(8, 64, 3, padding = 1)
        self.conv3 = nn.Conv1d(64, 512, 3, padding = 1)

        self.pool = nn.MaxPool1d(2, 2)    # kernel size, stride

        self.fc1 = nn.Linear(5632, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x


model = CNN()

To train the autoencoder, we use MSELoss and an Adam optimizer with learning rate 0.001.

In [24]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

We define our training function. Note that we ignore `y` for training the autoencoder, as we are trying to compress the features.

In [25]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [26]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Now, we train the autoencoder.

In [27]:
%%time
%%memit

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


loss: 0.686249  [    0/ 3647]
Test Error: 
 Accuracy: 80.5%, Avg loss: 0.508053 

Epoch 2
-------------------------------
loss: 0.563262  [    0/ 3647]
Test Error: 
 Accuracy: 80.5%, Avg loss: 0.508053 

Epoch 3
-------------------------------
loss: 0.563262  [    0/ 3647]
Test Error: 
 Accuracy: 80.5%, Avg loss: 0.508053 

Epoch 4
-------------------------------
loss: 0.563262  [    0/ 3647]
Test Error: 
 Accuracy: 80.5%, Avg loss: 0.508053 

Epoch 5
-------------------------------
loss: 0.563262  [    0/ 3647]
Test Error: 
 Accuracy: 80.5%, Avg loss: 0.508053 

Epoch 6
-------------------------------
loss: 0.563262  [    0/ 3647]
Test Error: 
 Accuracy: 80.5%, Avg loss: 0.508053 

Epoch 7
-------------------------------
loss: 0.563262  [    0/ 3647]
Test Error: 
 Accuracy: 80.5%, Avg loss: 0.508053 

Epoch 8
-------------------------------
loss: 0.563262  [    0/ 3647]
Test Error: 
 Accuracy: 80.5%, Avg loss: 0.508053 

Epoch 9
-------------------------------
loss: 0.563262  [    0/ 